In [0]:
dbutils.fs.mount(
  source = "wasbs://rafi@kalyanjsp.blob.core.windows.net",
  mount_point = "/mnt/dbfs",
  extra_configs ={"fs.azure.account.key.kalyanjsp.blob.core.windows.net":'2nNUUtf+WKbzpAjvQww/l8jVwB/Tx957GpI/o/vKkFuzNiupYklekGEP7UNJi0CfUyjbYhzN/8DT+ASt4uhONw=='})

Out[1]: True

In [0]:
circuit_df = spark.read.csv('dbfs:/mnt/dbfs/raw/circuits.csv',header = True,inferSchema=True)

In [0]:
from pyspark.sql.functions import current_timestamp
circuit_df2 = circuit_df.withColumnRenamed('circuitId','circuit_ID')\
                        .withColumnRenamed('circuitRef','circuit_Ref')\
                        .withColumnRenamed('lat','lattitude')\
                        .withColumnRenamed('lng','longitude')\
                        .withColumnRenamed('alt','altitude')\
                        .drop('url')\
                        .withColumn('time_stamp',current_timestamp())
circuit_df2.show()
circuit_df2.printSchema()

+----------+--------------+--------------------+------------+---------+---------+---------+--------+--------------------+
|circuit_ID|   circuit_Ref|                name|    location|  country|lattitude|longitude|altitude|          time_stamp|
+----------+--------------+--------------------+------------+---------+---------+---------+--------+--------------------+
|         1|   albert_park|Albert Park Grand...|   Melbourne|Australia| -37.8497|  144.968|      10|2023-01-27 10:24:...|
|         2|        sepang|Sepang Internatio...|Kuala Lumpur| Malaysia|  2.76083|  101.738|      18|2023-01-27 10:24:...|
|         3|       bahrain|Bahrain Internati...|      Sakhir|  Bahrain|  26.0325|  50.5106|       7|2023-01-27 10:24:...|
|         4|     catalunya|Circuit de Barcel...|    Montmeló|    Spain|    41.57|  2.26111|     109|2023-01-27 10:24:...|
|         5|      istanbul|       Istanbul Park|    Istanbul|   Turkey|  40.9517|   29.405|     130|2023-01-27 10:24:...|
|         6|        mona

In [0]:
circuit_df2.write.mode('overwrite').parquet('dbfs:/mnt/dbfs/silver/')

In [0]:
%fs ls '/mnt/dbfs/raw/races.csv'

path,name,size,modificationTime
dbfs:/mnt/dbfs/raw/races.csv,races.csv,116847,1674805737000


In [0]:
race_df = spark.read.csv('dbfs:/mnt/dbfs/raw/races.csv',header = True,inferSchema=True)
race_df.show()

+------+----+-----+---------+--------------------+----------+--------+--------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|
+------+----+-----+---------+--------------------+----------+--------+--------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|
|     3|2009|    3|       17|  Chinese Grand Prix|2009-04-19|07:00:00|http://en.wikiped...|
|     4|2009|    4|        3|  Bahrain Grand Prix|2009-04-26|12:00:00|http://en.wikiped...|
|     5|2009|    5|        4|  Spanish Grand Prix|2009-05-10|12:00:00|http://en.wikiped...|
|     6|2009|    6|        6|   Monaco Grand Prix|2009-05-24|12:00:00|http://en.wikiped...|
|     7|2009|    7|        5|  Turkish Grand Prix|2009-06-07|12:00:00|http://en.wikiped...|
|     8|2009|    8|        9|  British Grand Prix|2009-06-21|12:00:00|http://en.

In [0]:
from pyspark.sql import functions as sf
# from pyspark.sql.functions import concat,col,lit
races_df2 = race_df.withColumnRenamed('raceId','race_Id')\
                        .withColumnRenamed('circuitId','circuit_Id')\
                        .withColumnRenamed('year','race_year')\
                        .withColumn('Race_timestamp',sf.concat(sf.col('date'), sf.lit(' '),sf.col('time')))\
                        .withColumn('Ingestion_date',sf.current_timestamp())\
                        .drop('url','date','time')
races_df2.show()
races_df2.printSchema()

+-------+---------+-----+----------+--------------------+-------------------+--------------------+
|race_Id|race_year|round|circuit_Id|                name|     Race_timestamp|      Ingestion_date|
+-------+---------+-----+----------+--------------------+-------------------+--------------------+
|      1|     2009|    1|         1|Australian Grand ...|2009-03-29 06:00:00|2023-01-27 11:04:...|
|      2|     2009|    2|         2|Malaysian Grand Prix|2009-04-05 09:00:00|2023-01-27 11:04:...|
|      3|     2009|    3|        17|  Chinese Grand Prix|2009-04-19 07:00:00|2023-01-27 11:04:...|
|      4|     2009|    4|         3|  Bahrain Grand Prix|2009-04-26 12:00:00|2023-01-27 11:04:...|
|      5|     2009|    5|         4|  Spanish Grand Prix|2009-05-10 12:00:00|2023-01-27 11:04:...|
|      6|     2009|    6|         6|   Monaco Grand Prix|2009-05-24 12:00:00|2023-01-27 11:04:...|
|      7|     2009|    7|         5|  Turkish Grand Prix|2009-06-07 12:00:00|2023-01-27 11:04:...|
|      8| 

In [0]:
races_df2.write.option('header',True)\
                .partitionBy("race_year")\
                .mode('overwrite')\
                .csv('dbfs:/mnt/dbfs/silver/')